# Получение данных из Logs API и работа с ними

Предполагаю, что вы уже прочитали теоретическую часть в статье "Большие данные совсем близко: руководство по самостоятельной аналитике логов Яндекс.Метрики", если же нет - то советую с этого начать. Тем не менее, основные нюансы здесь будут продублированы и в данном блокноте.

Итак, для начала подгрузим необходимые библиотеки и настроим отображение графиков прямо в блокноте.

In [2]:
import pandas as pd
import requests
import json
import warnings
warnings.simplefilter('ignore')
%matplotlib inline

Теперь авторизуем наше приложение в аккаунте Яндекс.Метрики, перейдя по адресу вида https://oauth.yandex.ru/authorize?response_type=token&client_id={my_app_id} (где {my_app_id} - это id вашего приложения) и разрешим приложению работать с нужным аккаунтом Яндекс.Метрики

Целью этих действий является получения токена авторизации - XXXXXXXX

Напишем несколько функций, которые потом будем использовать для работы с Logs API

In [23]:
def evaluate_request(counter_id, date1, date2, fields, source, access_token):
    url = "https://api-metrika.yandex.ru/management/v1/counter/" + counter_id + \
"/logrequests/evaluate?date1=" + date1 + "&date2=" + date2 + "&fields=" + fields + \
"&source=" + source + "&oauth_token=" + access_token
    result = requests.get(url)
    result = result.text.encode('utf-8').decode('utf-8')
    result_dict = json.loads(result)
    return result_dict

def create_request(counter_id, date1, date2, fields, source, access_token):
    url = "https://api-metrika.yandex.ru/management/v1/counter/" + counter_id + \
"/logrequests?date1=" + date1 + "&date2=" + date2 + "&fields=" + fields + \
"&source=" + source + "&oauth_token=" + access_token
    result = requests.post(url)
    result = result.text.encode('utf-8').decode('utf-8')
    result_dict = json.loads(result)
    request_id = str(result_dict["log_request"]["request_id"])
    return request_id

def check_request(counter_id, request_id, access_token):
    url = "https://api-metrika.yandex.ru/management/v1/counter/" + counter_id + \
"/logrequest/" + request_id + "?&oauth_token=" + access_token
    result = requests.get(url)
    result = result.text.encode('utf-8').decode('utf-8')
    result = json.loads(result)
    return result

def download_logs():
    pass

Функции готовы, теперь зададим значения для их использования. Здесь вам нужно будет задать свои значения для переменных:

* counter_id - напишите в кавычках номер счётчика Яндекс.Метрики, по которому вы хотите получить информацию
* date1, date2 - здесь нужно будет написать начальную и конечную даты (в формате ГГГГ-ММ-ДД, также в кавычках)  того промежутка времени, за который вы хотите получить логи
* fields - здесь нужно будет перечислить те данные из логов, которые вы хотите получить. В принципе это поле можно не изменять, но можно и добавить/убрать какие-то поля. Полный список полей: https://tech.yandex.ru/metrika/doc/api2/logs/fields/visits-docpage/
* source - здесь нужно выбрать либо "visits", либо "hits", в зависимости от того, из какого источника вы хотите получить логи. В дальнейшем работа будет описана для визитов, но при желании вы можете запросить и данные на основе просмотров
* access_token - здесь нужно будет написать ваш токен авторизации, подробная инструкция о том, как его получить, написана в статье.

In [15]:
counter_id = "{my_counter_id}"
date1 = "2017-08-01"
date2 = "2017-08-31"
fields = "ym:s:visitID,ym:s:counterID,ym:s:watchIDs,ym:s:date,ym:s:dateTime,ym:s:isNewUser,ym:s:startURL," \
    "ym:s:endURL,ym:s:pageViews,ym:s:visitDuration,ym:s:bounce,ym:s:ipAddress,ym:s:params,ym:s:goalsID," \
    "ym:s:goalsSerialNumber,ym:s:goalsDateTime,ym:s:goalsPrice,ym:s:goalsOrder,ym:s:goalsCurrency,ym:s:clientID," \
    "ym:s:lastTrafficSource,ym:s:lastAdvEngine,ym:s:lastReferalSource,ym:s:lastSearchEngineRoot,ym:s:lastSearchEngine," \
    "ym:s:lastSocialNetwork,ym:s:lastSocialNetworkProfile,ym:s:referer,ym:s:lastDirectClickOrder,ym:s:lastDirectBannerGroup," \
    "ym:s:lastDirectClickBanner,ym:s:lastDirectPhraseOrCond,ym:s:lastDirectPlatformType,ym:s:lastDirectPlatform," \
    "ym:s:lastDirectConditionType,ym:s:lastCurrencyID,ym:s:from,ym:s:UTMCampaign,ym:s:UTMContent,ym:s:UTMMedium," \
    "ym:s:UTMSource,ym:s:UTMTerm,ym:s:regionCountry,ym:s:regionCity,ym:s:browserLanguage,ym:s:browserCountry," \
    "ym:s:clientTimeZone,ym:s:deviceCategory,ym:s:mobilePhone,ym:s:mobilePhoneModel,ym:s:operatingSystemRoot," \
    "ym:s:operatingSystem,ym:s:browser,ym:s:browserMajorVersion,ym:s:browserMinorVersion,ym:s:browserEngine," \
    "ym:s:cookieEnabled,ym:s:javascriptEnabled,ym:s:screenFormat,ym:s:screenColors,ym:s:screenOrientation," \
    "ym:s:screenWidth,ym:s:screenHeight,ym:s:physicalScreenWidth,ym:s:physicalScreenHeight,ym:s:windowClientWidth," \
    "ym:s:windowClientHeight,ym:s:networkType"
source = "visits"
access_token = "{my_access_token}"

Проверим возможность получения отчёта

In [18]:
evaluate_result = evaluate_request(counter_id = counter_id, date1=date1, date2=date2, fields=fields, source=source, access_token=access_token)
print evaluate_result

{u'log_request_evaluation': {u'max_possible_day_quantity': 52276, u'possible': True}}


В ответе мы видим, возможно ли получить отчёт и примерное максимально возможное кол-во дней, за которое можно будет получить логи.
Убедившись, что запрос можно создать, сделаем непосредственно сам запрос на получение отчёта

In [20]:
request_id = create_request(counter_id = counter_id, date1=date1, date2=date2, fields=fields, source=source, access_token=access_token)

In [21]:
request_id

'228219'

Запрос сделали и получили его id, теперь проверим его статус (готов ли он или нет). Проверяем, пока не увидим значение у поля status - processed.

In [26]:
status = check_request(counter_id=counter_id, request_id=request_id, access_token=access_token)

In [27]:
status

{u'log_request': {u'counter_id': 26756901,
  u'date1': u'2017-08-01',
  u'date2': u'2017-08-31',
  u'fields': [u'ym:s:visitID',
   u'ym:s:counterID',
   u'ym:s:watchIDs',
   u'ym:s:date',
   u'ym:s:dateTime',
   u'ym:s:isNewUser',
   u'ym:s:startURL',
   u'ym:s:endURL',
   u'ym:s:pageViews',
   u'ym:s:visitDuration',
   u'ym:s:bounce',
   u'ym:s:ipAddress',
   u'ym:s:params',
   u'ym:s:goalsID',
   u'ym:s:goalsSerialNumber',
   u'ym:s:goalsDateTime',
   u'ym:s:goalsPrice',
   u'ym:s:goalsOrder',
   u'ym:s:goalsCurrency',
   u'ym:s:clientID',
   u'ym:s:lastTrafficSource',
   u'ym:s:lastAdvEngine',
   u'ym:s:lastReferalSource',
   u'ym:s:lastSearchEngineRoot',
   u'ym:s:lastSearchEngine',
   u'ym:s:lastSocialNetwork',
   u'ym:s:lastSocialNetworkProfile',
   u'ym:s:referer',
   u'ym:s:lastDirectClickOrder',
   u'ym:s:lastDirectBannerGroup',
   u'ym:s:lastDirectClickBanner',
   u'ym:s:lastDirectPhraseOrCond',
   u'ym:s:lastDirectPlatformType',
   u'ym:s:lastDirectPlatform',
   u'ym:s:lastD